# HOOMD FFS API Test - April 28th 2022

In [1]:
import numpy as np

import matplotlib.pyplot as plt

from numba import njit, vectorize, float32

from monk import nb

from typing import Callable, Optional

import hoomd
import hoomd.forward_flux.forward_flux as ffs
import gsd.hoomd

In [ ]:
cpu = hoomd.device.CPU()
seed = 1234
sim = ffs.ForwardFluxSimulation(cpu, 0, seed=seed)